In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.offline as pyo
import torch
import sys

sys.path.append("../anomalyGAN/code")

from anomalyGAN import *
from anomalyDetector_lib import *

In [3]:
def extract_anomal_data(time_series, meta_information, length, skip_points=1):
    
    complete_length = time_series.shape[0]
    
    return_array = []
    
    return_meta_information = []
    
    for i in range(0, int((complete_length-length)/skip_points)):
        
        return_array.append(time_series[i:i+length])
        
        return_meta_information.append(meta_information[i:i+length])
        
        
    return np.array(return_array), np.array(return_meta_information)

def load_cdn_data(sequence_length=256):
    
    aCDN = np.load("../data/data.npy")
    anomalyInformation = np.load("../data/condInput.npy")

    def sampleAnomalyC(batch_size):
        return torch.Tensor(aMetaInformation[np.random.randint(0,aMetaInformation.shape[0],size=batch_size)])
    
    aData, aMetaInformation = extract_anomal_data(aCDN, anomalyInformation, sequence_length)

    return aData, aMetaInformation, sampleAnomalyC

In [4]:
aData, aMetaInformation, sampleAnomalyC = load_cdn_data(256)

In [5]:
print(f"aData.shape: {aData.shape}")
print(f"aMetaInformation.shape: {aMetaInformation.shape}")

aData.shape: (19472, 256, 3)
aMetaInformation.shape: (19472, 256, 1)


In [6]:
anomal_model_config = {
    "d_steps": 1,
    "g_steps": 3,
    "lr": 0.0001,
    "wgan": False,
    "sampleC_fun": sampleAnomalyC,
    "optim": "ADAM",
    "batch_size": 256,
    "architecture": ["TCN", "TCN"],
    "Generator": {
        "channels": 100,
        "num_layers": 8,
        "kernel_size": 5,
        "dropout": 0.25        
    },
    "Discriminator": {
        "channels": 100,
        "num_layers": 8,
        "kernel_size": 5,
        "dropout": 0.25        
    },
    "scale": True,
    "z_latent_dim": 3,
    "num_fixed_noises": 200,
    "patience": 150
}

training_data_config = {
    "num_data_points": 256,
    "z_seq_length": 256,
    "multi_variates": 3,
    "num_anomalies": 0
}


In [7]:
training_data_config = {
        "num_data_points": 256,
        "z_seq_length": 256,
        "multi_variates": 3,
        "num_frequencies": 4,
        "num_anomalies": 0
}

In [8]:
detector_path = "./"
ad_training_data_config = json.load( open( f"{detector_path}/training_data_config.json" ) )
ad_tcn_model_config = json.load( open( f"{detector_path}/model_config.json" ) )

anomalyDetector = AnomalyDetector(ad_training_data_config,ad_tcn_model_config,None)

pre_trained_model = torch.load(f"{detector_path}/best_model.pt")

anomalyDetector.model = pre_trained_model

Device: cuda:0


In [9]:
anomalyGAN = AnomalyGAN(training_data_config, anomal_model_config, aData[:1000],aMetaInformation[:1000],anomalyDetector)

In [ ]:
anomalyGAN.train(25,use_wandb=False, print_iterations=1,save_path=None)

cuda:0
Starting Training Loop...
Epoch 0
Using Threshold == 0.018666400715257183
Validation loss decreased (inf --> 0.511545).  Saving model ...
{'f1': 0.06591064470968742, 'precision': 0.034078386, 'recall': 1.0, 'auc_score': 0.5115445910512857, 'mse': 0.06215404, 'FFT-Difference': 0.010627285146134851}
Epoch 1


In [ ]:
anomalyGAN.visualize_training()